In [1]:
pip install selenium beautifulsoup4 pandas

  Using cached urllib3-2.4.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached websocket_client-1.8.0-py3-none-any.whl.metadata (8.0 kB)
     ---------------------------------------- 0.0/60.9 kB ? eta -:--:--
     ---------------------------------------- 60.9/60.9 kB 1.1 MB/s eta 0:00:00
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
   ---------------------------------------- 0.0/9.4 MB ? eta -:--:--
    --------------------------------------- 0.2/9.4 MB 4.6 MB/s eta 0:00:03
   - -------------------------------------- 0.3/9.4 MB 3.2 MB/s eta 0:00:03
   - -------------------------------------- 0.4/9.4 MB 3.8 MB/s eta 0:00:03
   -- ----------------------


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service
# from selenium.webdriver.chrome.options import Options

# # Ubah path ini sesuai lokasi file chromedriver kamu ya!
# path_to_driver = "C:/chromedriver/chromedriver.exe"

# chrome_options = Options()
# chrome_options.add_argument("--start-maximized")
# chrome_options.add_argument("--disable-popup-blocking")
# chrome_options.add_argument("--disable-infobars")
# chrome_options.add_argument("--disable-notifications")

# # Setup WebDriver
# driver = webdriver.Chrome(service=Service(path_to_driver), options=chrome_options)

# # Coba buka Google dulu
# driver.get("https://www.google.com")


In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
import time, os, re
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
# Setup
path_to_driver = r"C:/chromedriver/chromedriver.exe"
email = ""
password = ""
csv_path = "job_links_glints.csv"

In [5]:
# Chrome Options
chrome_options = Options()
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("--disable-notifications")
driver = webdriver.Chrome(service=Service(path_to_driver), options=chrome_options)

# ---------- LOGIN FLOW ----------
def login():
    driver.get("https://glints.com/id")
    time.sleep(5)

    driver.find_element(By.XPATH, '//button[span[text()="Masuk"]]').click()
    time.sleep(2)
    driver.find_element(By.XPATH, '//a[span[text()="Masuk dengan Email"]]').click()
    time.sleep(2)

    driver.find_element(By.ID, "login-form-email").send_keys(email)
    driver.find_element(By.ID, "login-form-password").send_keys(password)
    print("Login info filled")

    driver.find_element(By.XPATH, '//button[@data-cy="submit_btn_login"]').click()
    time.sleep(5)
    print("Logged in successfully")

login()

# ---------- SCRAPE FLOW ----------
def scrape_jobs(job_keyword):
    print(f"\nScraping for: {job_keyword}")

    # Klik logo balik ke home
    try:
        driver.find_element(By.CLASS_NAME, "sKTdp").click()
        time.sleep(2)
    except:
        print("Gagal klik logo, skip balik home")

    # Isi pencarian
    search_input = driver.find_element(By.XPATH, '//input[@placeholder="Cari Nama Pekerjaan, Skill, dan Perusahaan"]')
    search_input.clear()
    search_input.send_keys(job_keyword)
    search_input.send_keys(Keys.ENTER)
    time.sleep(5)

    # Scroll load semua
    prev_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        try:
            driver.find_element(By.XPATH, '//button[@data-testid="close-button"]').click()
            print("Pop-up login muncul lagi, ditutup")
        except:
            pass
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == prev_height:
            break
        prev_height = new_height

    # Ambil link lowongan
    soup = BeautifulSoup(driver.page_source, "html.parser")
    job_cards = soup.find_all("a", href=re.compile("^/id/opportunities/jobs/"))
    new_links = ["https://glints.com" + a["href"] for a in job_cards]
    new_links = list(dict.fromkeys(new_links))

    print(f"Ditemukan {len(new_links)} link untuk {job_keyword}")

    # Baca file sebelumnya
    if os.path.exists(csv_path):
        old_df = pd.read_csv(csv_path)
        old_links = old_df["Job Link"].tolist()
    else:
        old_links = []

    # Gabung & simpan
    combined_links = list(dict.fromkeys(old_links + new_links))
    df = pd.DataFrame(combined_links, columns=["Job Link"])
    df.to_csv(csv_path, index=False, encoding="utf-8-sig")
    print(f"Total {len(combined_links)} link tersimpan di '{csv_path}'")

# ---------- ITERASI KEYWORD ----------
job_keywords = [
    # Tech & Data
    "Full Stack Developer",
    "Mobile Developer", "Data Scientist", "Machine Learning", "Cyber Security",
    "DevOps Engineer", "QA Engineer", "IT Support", "Database Administrator", "Product Manager",

    # UI/UX & Creative
    "UI Designer", "UX Designer", "Graphic Designer", "Creative Designer", "Motion Designer",
    "Illustrator", "Video Editor", "Animator", "3D Artist", "Content Creator",

    # Business & Management
    "Business Analyst", "Project Manager", "Business Development", "Operations Manager",
    "Sales Executive", "Account Executive", "Partnership Manager", "Consultant",
    "Product Owner", "Scrum Master", "Strategy Analyst",

    # Marketing & Communications
    "Digital Marketing", "Content Writer", "Copywriter", "SEO Specialist", "Marketing Strategist",
    "Social Media Specialist", "PR Specialist", "Brand Manager", "Market Research",

    # Finance & Legal
    "Finance Officer", "Accounting Staff", "Tax Consultant", "Auditor", "Financial Analyst",
    "Investment Analyst", "Risk Analyst", "Legal Officer", "Corporate Secretary", "Compliance Officer",

    # HR & GA
    "HR Generalist", "HRBP", "Talent Acquisition", "Recruiter", "People Development",
    "Training Specialist", "Payroll Staff", "Office Manager", "GA Staff",

    # Education & Training
    "Teacher", "Private Tutor", "Curriculum Developer", "Instructional Designer", 
    "Education Consultant", "Trainer", "Mentor", "Counselor",

    # Healthcare & Labs
    "Nurse", "Doctor", "Pharmacist", "Nutritionist", "Medical Staff", "Lab Analyst",
    "Health Analyst", "Radiographer", "Clinical Researcher",

    # F&B & Hospitality
    "Chef", "Barista", "Cook", "Waiter", "Kitchen Staff", "Bartender",
    "Hotel Receptionist", "Housekeeping", "Event Organizer",

    # Manufacturing & Logistics
    "Production Staff", "Warehouse Staff", "Procurement", "Quality Control",
    "Logistics", "Supply Chain", "Maintenance Engineer", "Mechanical Engineer",

    # Design & Architecture
    "Interior Designer", "Architect", "Draftsman", "Civil Engineer", "Urban Planner",

    # Customer Support
    "Customer Service", "Call Center", "Telemarketing", "Client Support", "Technical Support",

    # Entry-level & Internship
    "Intern", "Magang", "Fresh Graduate", "Admin", "Office Assistant", "Staff",
    "Receptionist", "Data Entry", "Junior Staff", "Student Intern"
]

for keyword in job_keywords:
    scrape_jobs(keyword)
    time.sleep(3)

driver.quit()

Login info filled
Logged in successfully

Scraping for: Full Stack Developer
Ditemukan 991 link untuk Full Stack Developer
Total 4271 link tersimpan di 'job_links_glints.csv'

Scraping for: Mobile Developer
Ditemukan 991 link untuk Mobile Developer
Total 5261 link tersimpan di 'job_links_glints.csv'

Scraping for: Data Scientist
Ditemukan 882 link untuk Data Scientist
Total 6142 link tersimpan di 'job_links_glints.csv'

Scraping for: Machine Learning
Ditemukan 846 link untuk Machine Learning
Total 6987 link tersimpan di 'job_links_glints.csv'

Scraping for: Cyber Security
Ditemukan 586 link untuk Cyber Security
Total 7572 link tersimpan di 'job_links_glints.csv'

Scraping for: DevOps Engineer
Ditemukan 991 link untuk DevOps Engineer
Total 8562 link tersimpan di 'job_links_glints.csv'

Scraping for: QA Engineer
Ditemukan 991 link untuk QA Engineer
Total 9552 link tersimpan di 'job_links_glints.csv'

Scraping for: IT Support
Ditemukan 991 link untuk IT Support
Total 10542 link tersimpan 

InvalidSessionIdException: Message: invalid session id; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalidsessionidexception
Stacktrace:
	GetHandleVerifier [0x0x7ff6138c6f95+76917]
	GetHandleVerifier [0x0x7ff6138c6ff0+77008]
	(No symbol) [0x0x7ff613679c1c]
	(No symbol) [0x0x7ff6136c055f]
	(No symbol) [0x0x7ff6136f8332]
	(No symbol) [0x0x7ff6136f2e53]
	(No symbol) [0x0x7ff6136f1f19]
	(No symbol) [0x0x7ff613644b05]
	GetHandleVerifier [0x0x7ff613b9d2cd+3051437]
	GetHandleVerifier [0x0x7ff613b97923+3028483]
	GetHandleVerifier [0x0x7ff613bb58bd+3151261]
	GetHandleVerifier [0x0x7ff6138e185e+185662]
	GetHandleVerifier [0x0x7ff6138e971f+218111]
	(No symbol) [0x0x7ff613643b00]
	GetHandleVerifier [0x0x7ff613cb5f38+4201496]
	BaseThreadInitThunk [0x0x7ffe136d259d+29]
	RtlUserThreadStart [0x0x7ffe15a8af58+40]
